In [118]:
import pandas as pd
import numpy as np

EndDate = '2021-12-25'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq 
mylist = [['The Pond Rent   ',650,'2021-01-01','7D'],
          ['Wolli Creek Rent',450,'2021-01-01','7D'],
          ['Salary          ',450,'2021-01-01','7D'],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    date = pd.date_range(mylist[i][2],EndDate,freq=mylist[i][3])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date']).dt.to_period('M')

Total = df[['Period','Description','Amount']]
Total.groupby(['Period','Description']).sum().reset_index()


,Period,Description,Amount
0,2021-01,Salary,2250
1,2021-01,The Pond Rent,3250
2,2021-01,Wolli Creek Rent,2250
3,2021-02,Salary,1800
4,2021-02,The Pond Rent,2600
5,2021-02,Wolli Creek Rent,1800
6,2021-03,Salary,1800
7,2021-03,The Pond Rent,2600
8,2021-03,Wolli Creek Rent,1800
9,2021-04,Salary,2250


In [196]:

def cat(mytext):
    
    # list of keywords, add as you find new ones
    WolliRent        = ['slick','carena']
    ThePondRent      = ['harcourts','hills Prestige','hillside']
    Grocery          = ['Coles','woolworths','woolworth','tongli','WW Metro','supermarket','iga']
    Entertainment    = ['rebel','chemist','mcdonald','malatang','kmart','chatime','kogan','tea','indo'
                       ,'pondok','bread','subway','gong','chinese','kfc','mie','ciak','japanese','thai',
                       'saigon','bakery','racha','taste']
    Salary           = ['salary']
    Utilities        = ['TPG','ENERGY','AGL','Netflix','OPTUS']
    Insurance        = ['life','ins','medibank']
    Excluded         = ['Payment']
    Car              = ['Top One','AAMI','BP']
    # combine all the categories into one list called collection
    collection      = [WolliRent,ThePondRent,Grocery,Entertainment,Salary,Utilities,Insurance,Excluded,Car]


    for i in range(len(collection)):
        for k in range(len(collection[i])):
            if collection[i][k].lower() in mytext.lower():
                if i == 0:
                    cat = 'Rent - Wolli'
                elif i == 1:
                    cat = 'Rent - The Pond'
                elif i == 2:
                    cat = 'Grocery'
                elif i == 3:
                    cat = 'Entertainment'
                elif i == 4:
                    cat = 'Salary'
                elif i == 5:
                    cat = 'Utilities'
                elif i == 6:
                    cat = 'Insurance'
                elif i == 7:
                    cat = 'Excluded'
                elif i == 8:
                    cat = 'Car'   
                return cat
            
   

In [197]:
import pandas as pd
cba = pd.read_csv(r'C:\Users\m118954\mydsbook\Data\CSVData.csv', header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Description']


cba['Cat'] = cba.apply(lambda x:cat(x['Description']),axis=1)
cba['Cat'].fillna('Other',inplace=True)

cba.to_csv(r'C:\Users\m118954\mydsbook\Data\CSVData2.csv')